In [1]:
import pandas as pd
import datetime
import requests

In [2]:
sites_data = pd.read_csv('site_data.csv')
sites_data.head()

,id,azimuth,tilt,type,capacity(kWp),lat,lon
0,Downtown,168,15,poli,4.68,19.423630,-99.131770
1,West,193,11,poli,5.83,19.399021,-99.245410
2,East,180,13,poli,2.38,19.404850,-98.992381
3,South,135,13,poli,2.65,19.278835,-99.142413
4,North,197,13,poli,3.18,19.560209,-99.127307


In [3]:
ids = sites_data['id']
ids = ids.to_list()
ids

['Downtown', 'West', 'East', 'South', 'North']

In [4]:
latitud = sites_data['lat']
latitud = latitud.to_list()
latitud

[19.42363, 19.399020999999998, 19.40485, 19.278835, 19.560209]

In [5]:
longitud = sites_data['lon']
longitud = longitud.to_list()
longitud

[-99.13176999999999,
 -99.24540999999999,
 -98.99238100000001,
 -99.142413,
 -99.127307]

In [6]:
calendar = pd.read_excel('calendar.xlsx')
#calendar = calendar.astype({"dates": object, "end_date": object})

In [7]:
calendar["dates"] = calendar["dates"].map(lambda ts: ts.strftime("%Y-%m-%d"))
calendar["end_date"] = calendar["end_date"].map(lambda ts: ts.strftime("%Y-%m-%d"))

In [8]:
calendar.info()
calendar.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 2 columns):
dates       12 non-null object
end_date    12 non-null object
dtypes: object(2)
memory usage: 320.0+ bytes


,dates,end_date
0,2019-08-01,2019-08-31
1,2019-09-01,2019-09-30
2,2019-10-01,2019-10-31
3,2019-11-01,2019-11-30
4,2019-12-01,2019-12-31


In [9]:
dates = calendar['dates']
dates = dates.to_list()
dates

['2019-08-01',
 '2019-09-01',
 '2019-10-01',
 '2019-11-01',
 '2019-12-01',
 '2020-01-01',
 '2020-02-01',
 '2020-03-01',
 '2020-04-01',
 '2020-05-01',
 '2020-06-01',
 '2020-07-01']

In [10]:
end_date = calendar['end_date']
end_date = end_date.to_list()
end_date

['2019-08-31',
 '2019-09-30',
 '2019-10-31',
 '2019-11-30',
 '2019-12-31',
 '2020-01-31',
 '2020-02-29',
 '2020-03-31',
 '2020-04-30',
 '2020-05-31',
 '2020-06-30',
 '2020-07-31']

In [12]:
key = '5f813d72e7854f6caa1151047200108'

# List of dates corresponding to the month and the year (The API only manages to get a month at a time)
# dates = ['2019-01-01', '2019-02-01', '2019-03-01', '2019-04-01', '2019-05-01', '2019-06-01', '2019-07-01', '2019-08-01', '2019-09-01', '2019-10-01', '2019-11-01', '2019-12-01', '2020-01-01', '2020-02-01', '2020-03-01', '2020-04-01', '2020-05-01']
# end_date = ['2019-01-31', '2019-02-28', '2019-03-31', '2019-04-30', '2019-05-31', '2019-06-30', '2019-07-31', '2019-08-31', '2019-09-30', '2019-10-31', '2019-11-30', '2019-12-31', '2020-01-31', '2020-02-29', '2020-03-31', '2020-04-30', '2020-05-31']
# Create an empty list to store all the data
ids_list = []
date_list = []
lat_column = []
lon_column = []
UVIndex_list = []
sunHour_list = []
cloud_list = []
MaxtempC_list = []
MintempC_list = []
sunrise_list = []
sunset_list = []
precMM_list = []


# Latitude and Longitude of the sites

site_flag = 0

for lat in latitud:

    end_date_flag = 0
    # For each start and end date, get the URL. Use a flag to iterate through the end_date list.
    for date in dates:
        # Built the URL.
        url='http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key={key}&q={latitud},%{longitud}&format=json&date={date}&enddate={end_date}&includelocation=yes&tp=24'.format(
        key=key,latitud=lat,longitud=longitud[site_flag],date=date,end_date=end_date[end_date_flag])
        end_date_flag+=1
        # Get response from the URL.
        response = requests.get(url).json()
        # Create a flag to iterate thourgh the hole month.
        month_day = 0
        # For each element in the JSON (day of the mont), extract the Date, Avg Temperature C,
        # Humidity %, Feels Like C.
        for i in response['data']['weather']:
            # Append the element of the dictionary that corresponds to each list. 
            date_list.append(response['data']['weather'][month_day]['date'])
            MaxtempC_list.append(float(response['data']['weather'][month_day]['maxtempC']))
            MintempC_list.append(float(response['data']['weather'][month_day]['mintempC']))
            UVIndex_list.append(float(response['data']['weather'][month_day]['uvIndex']))
            sunHour_list.append(float(response['data']['weather'][month_day]['sunHour']))
            cloud_list.append(float(response['data']['weather'][month_day]['hourly'][0]['cloudcover']))
            precMM_list.append(float(response['data']['weather'][month_day]['hourly'][0]['precipMM']))
            sunrise_list.append(str(response['data']['weather'][month_day]['astronomy'][0]['sunrise']))
            sunset_list.append(str(response['data']['weather'][month_day]['astronomy'][0]['sunset']))
            ids_list.append(ids[site_flag])
            month_day+=1

    # Create a dictionary to store the lists
    dict_climate = {'date':date_list, 
                    'id': ids_list,
                    'uvIndex': UVIndex_list,
                    'sunHour': sunHour_list,
                    'cloudcover': cloud_list,
                    'maxtempC': MaxtempC_list,
                    'mintempC': MintempC_list,
                    'precipitation': precMM_list,
                    'sunrise': sunrise_list,
                    'sunset': sunset_list
                   }
    print(str(lat) + ', ' + str(longitud[site_flag]))
    site_flag+=1

# Create the DataFrame based on the previous dictionary
df_climate = pd.DataFrame(dict_climate)
df_climate['date'] = pd.to_datetime(df_climate['date'])
df_climate.head()

19.42363, -99.13176999999999
19.399020999999998, -99.24540999999999
19.40485, -98.99238100000001
19.278835, -99.142413
19.560209, -99.127307


,date,id,uvIndex,sunHour,cloudcover,maxtempC,mintempC,precipitation,sunrise,sunset
0,2019-08-01,Downtown,4.0,10.3,42.0,24.0,9.0,1.7,07:13 AM,08:13 PM
1,2019-08-02,Downtown,4.0,7.8,49.0,24.0,10.0,4.7,07:13 AM,08:12 PM
2,2019-08-03,Downtown,5.0,11.6,36.0,25.0,10.0,2.1,07:14 AM,08:12 PM
3,2019-08-04,Downtown,4.0,7.8,56.0,24.0,11.0,15.7,07:14 AM,08:11 PM
4,2019-08-05,Downtown,4.0,6.5,77.0,23.0,10.0,25.1,07:14 AM,08:11 PM


In [13]:
df_climate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1830 entries, 0 to 1829
Data columns (total 10 columns):
date             1830 non-null datetime64[ns]
id               1830 non-null object
uvIndex          1830 non-null float64
sunHour          1830 non-null float64
cloudcover       1830 non-null float64
maxtempC         1830 non-null float64
mintempC         1830 non-null float64
precipitation    1830 non-null float64
sunrise          1830 non-null object
sunset           1830 non-null object
dtypes: datetime64[ns](1), float64(6), object(3)
memory usage: 143.1+ KB


In [14]:
df_climate = pd.DataFrame(dict_climate)
df_climate['date'] = pd.to_datetime(df_climate['date'])
df_climate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1830 entries, 0 to 1829
Data columns (total 10 columns):
date             1830 non-null datetime64[ns]
id               1830 non-null object
uvIndex          1830 non-null float64
sunHour          1830 non-null float64
cloudcover       1830 non-null float64
maxtempC         1830 non-null float64
mintempC         1830 non-null float64
precipitation    1830 non-null float64
sunrise          1830 non-null object
sunset           1830 non-null object
dtypes: datetime64[ns](1), float64(6), object(3)
memory usage: 143.1+ KB


In [15]:
df_climate.to_csv("clima_predict.csv", encoding='utf-8', index=False)

In [26]:
df_pls = pd.read_csv('shinji_eva_final.csv')
df_pls

,date,id,uvIndex,sunHour,cloudcover,maxtempC,mintempC,precipitation,sunrise,sunset
0,2017-01-01,1,4.0,11.0,7.0,20.0,8.0,0.0,08:11 AM,07:10 PM
1,2017-01-02,1,4.0,11.0,2.0,21.0,8.0,0.0,08:11 AM,07:11 PM
2,2017-01-03,1,4.0,11.0,2.0,22.0,9.0,0.0,08:11 AM,07:12 PM
3,2017-01-04,1,4.0,11.0,2.0,23.0,9.0,0.0,08:11 AM,07:12 PM
4,2017-01-05,1,4.0,11.0,0.0,22.0,7.0,0.0,08:12 AM,07:13 PM
...,...,...,...,...,...,...,...,...,...,...
20923,2020-07-27,26,4.0,9.1,49.0,23.0,10.0,3.9,07:12 AM,08:15 PM
20924,2020-07-28,26,4.0,9.1,70.0,23.0,11.0,9.7,07:12 AM,08:14 PM
20925,2020-07-29,26,4.0,7.8,77.0,23.0,11.0,12.1,07:12 AM,08:14 PM
20926,2020-07-30,26,4.0,9.1,72.0,25.0,11.0,13.7,07:13 AM,08:13 PM


In [43]:
production = pd.read_csv('production.csv')
production.head()

,id,date,produccion(Wh)
0,1,27/05/2020,34047.254
1,1,28/05/2020,37065.480
2,1,29/05/2020,37592.620
3,1,30/05/2020,34483.457
4,1,31/05/2020,33053.957


In [44]:
production.dtypes

id                  int64
date               object
produccion(Wh)    float64
dtype: object

In [45]:
df_pls.dtypes

date             datetime64[ns]
id                        int64
uvIndex                 float64
sunHour                 float64
cloudcover              float64
maxtempC                float64
mintempC                float64
precipitation           float64
sunrise                  object
sunset                   object
dtype: object

In [46]:
production['date'] = pd.to_datetime(production['date'], format="%d/%m/%Y")
production.dtypes

id                         int64
date              datetime64[ns]
produccion(Wh)           float64
dtype: object

In [47]:
production.head()

,id,date,produccion(Wh)
0,1,2020-05-27,34047.254
1,1,2020-05-28,37065.480
2,1,2020-05-29,37592.620
3,1,2020-05-30,34483.457
4,1,2020-05-31,33053.957


In [28]:
df_pls['date'] = pd.to_datetime(df_pls['date'], yearfirst=True, infer_datetime_format=True)

In [30]:
df_pls.head()

,date,id,uvIndex,sunHour,cloudcover,maxtempC,mintempC,precipitation,sunrise,sunset
0,2017-01-01,1,4.0,11.0,7.0,20.0,8.0,0.0,08:11 AM,07:10 PM
1,2017-01-02,1,4.0,11.0,2.0,21.0,8.0,0.0,08:11 AM,07:11 PM
2,2017-01-03,1,4.0,11.0,2.0,22.0,9.0,0.0,08:11 AM,07:12 PM
3,2017-01-04,1,4.0,11.0,2.0,23.0,9.0,0.0,08:11 AM,07:12 PM
4,2017-01-05,1,4.0,11.0,0.0,22.0,7.0,0.0,08:12 AM,07:13 PM


In [53]:
df_final = pd.merge(production, df_pls, how='right', on=['id', 'date'])

In [55]:
df_final.to_csv('plsplsfinal.csv', index=False)